In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
gpu_num = 3
print('GPU USE:', gpu_num)
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_num) # specify gpu number

GPU USE: 3


In [ ]:
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, concatenate
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers, optimizers

In [1]:
import numpy as np
import pickle, json
from matplotlib import pyplot as plt
from IPython import display

In [11]:
from sklearn.model_selection import train_test_split

In [2]:
dataset_path = './dataset/perturbations/datasets/2/'

load data

In [3]:
dispersion_data_mat = np.loadtxt(dataset_path + 'dispersion_data_mat.txt', delimiter=',')
dlf = np.loadtxt(dataset_path + 'dlf.txt', delimiter=',')
sensitity_data = np.loadtxt(dataset_path + 'sensitity_data.txt', delimiter=',')
print(dispersion_data_mat.shape)
print(dlf.shape)
print(sensitity_data.shape)

(2000, 45)
(2000, 8)
(2000, 8)


In [4]:
assert dispersion_data_mat.shape[0] == dlf.shape[0]
assert dlf.shape[0] == sensitity_data.shape[0]
dataset_size = dispersion_data_mat.shape[0]
print('dataset_size:', dataset_size)

dataset_size: 2000


norilize data X

In [5]:
# calc mu
mu_curve = np.zeros([dispersion_data_mat.shape[1]])
for i in range(dataset_size):
    mu_curve[:] += dispersion_data_mat[i, :]
mu_curve /= dataset_size
# calc std
std_curve = np.zeros([dispersion_data_mat.shape[1]])
for i in range(dataset_size):
    std_curve += (dispersion_data_mat[i, :] - mu_curve)**2/dataset_size
std_curve = np.sqrt(std_curve)
std_curve[np.where(std_curve == 0)] = 1

In [6]:
# Standardization
data_x = np.copy(dispersion_data_mat)
for i in range(dataset_size):
    data_x[i, :] -= mu_curve
    data_x[i, :] /= std_curve

norilize data Y

In [7]:
data_y = np.copy(dlf)
max_param_value = []
mean_param_value = []
for i in range(data_y.shape[1]):
    # get max
    max_param_value.append(np.max(data_y[:,i]))
    # get mean
    mean_param_value.append(np.mean(data_y[:,i]))
for i in range(data_y.shape[1]): 
    # normalize mean
    data_y[:,i] -= mean_param_value[i]
    # normalize max
    data_y[:,i] /= max_param_value[i]

train test split

In [12]:
x_train, x_val, y_train, y_val = train_test_split(data_x, data_y, train_size=0.9)

NN construction

In [ ]:
main_input = Input(shape=(x_train.shape[1],), name = 'disp_curve_flat_stf0f1')
x = Dense(500, activation='relu')(main_input)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(500, activation='relu')(x)
main_output = Dense(y_train.shape[1], activation='linear')(x)
model = Model(inputs = main_input, outputs = main_output)
print('model initialized')
with open(train_output_folder + 'model_summary.txt', 'w') as file_write_sum:
    with redirect_stdout(file_write_sum):
        model.summary()
plot_model(model, to_file=train_output_folder + 'model.png')

Forward solution analysis

In [ ]:
train_output_folder = './train_output_forward/'

In [ ]:
with open('./dataset/clust_poly3_cut_20160.pkl', 'rb') as fp:
    clust_poly3_cut_20160 = pickle.load(fp)

In [ ]:
dataset_size = 20148

In [ ]:
dcurve_Ndots = 15
data_X = np.zeros([dataset_size, dcurve_Ndots*4])
for i in range(dataset_size):
    data_X[i, :dcurve_Ndots] = clust_poly3_cut_20160['mono']['1_mode_s'][i][:]
    data_X[i, dcurve_Ndots:dcurve_Ndots*2] = clust_poly3_cut_20160['mono']['2_mode_s'][i][:]
    data_X[i, dcurve_Ndots*2:dcurve_Ndots*3] = clust_poly3_cut_20160['dipo']['1_mode_s'][i][:]
    data_X[i, dcurve_Ndots*3:] = clust_poly3_cut_20160['dipo']['2_mode_s'][i][:]


coef1 = np.max(data_X[:, :dcurve_Ndots])
coef2 = np.max(data_X[:, dcurve_Ndots:dcurve_Ndots*2])
coef3 = np.max(data_X[:, dcurve_Ndots*2:dcurve_Ndots*3])
coef4 = np.max(data_X[:, dcurve_Ndots*3:])

In [ ]:
with open(train_output_folder + 'history', 'rb') as file_history:
    history = pickle.load(file_history)
with open(train_output_folder + 'predictions_valid.pkl', 'rb') as fp:
    predictions_valid = pickle.load(fp)
with open(train_output_folder + 'y_val.pkl', 'rb') as fp:
    y_val = pickle.load(fp)

In [ ]:
y_val.shape

In [ ]:
a = np.random.rand(3,2)

In [ ]:
a

In [ ]:
np.random.shuffle(a)

In [ ]:
a

In [ ]:
plt.plot(history['val_loss'][1000:])
plt.plot(history['loss'][1000:])

In [ ]:
y_val.shape

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
for i in range(0, 2015, 10):
    ax.cla()
    ax.plot(clust_poly3_cut_20160['mono']['1_mode_f'], y_val[i,:dcurve_Ndots]*coef1, 'ko', label='true')
    ax.plot(clust_poly3_cut_20160['mono']['2_mode_f'], y_val[i,dcurve_Ndots:dcurve_Ndots*2]*coef2, 'ko')
    ax.plot(clust_poly3_cut_20160['dipo']['1_mode_f'], y_val[i,dcurve_Ndots*2:dcurve_Ndots*3]*coef3, 'ko')
    ax.plot(clust_poly3_cut_20160['dipo']['2_mode_f'], y_val[i,dcurve_Ndots*3:]*coef4, 'ko')
    ax.plot(clust_poly3_cut_20160['mono']['1_mode_f'], predictions_valid[i, :dcurve_Ndots]*coef1, 'r.', label='pred')
    ax.plot(clust_poly3_cut_20160['mono']['2_mode_f'], predictions_valid[i, dcurve_Ndots:dcurve_Ndots*2]*coef2, 'r.')
    ax.plot(clust_poly3_cut_20160['dipo']['1_mode_f'], predictions_valid[i, dcurve_Ndots*2:dcurve_Ndots*3]*coef3, 'r.')
    ax.plot(clust_poly3_cut_20160['dipo']['2_mode_f'], predictions_valid[i, dcurve_Ndots*3:]*coef4, 'r.')
    ax.legend(loc='upper right')
    ax.set_ylabel('slow, musec/m')
    ax.set_xlabel('freq, kHz')
    display.display(fig)
    display.clear_output(wait = True)
    plt.pause(1)

Weight analysis

In [ ]:
import tensorflow as tf
mse = tf.keras.losses.MeanSquaredError()

In [ ]:
from tensorflow import keras.load_model
model1 = keras.models.load_model('./experiments3/1/train_output3/model.h5')
model2 = keras.models.load_model('./experiments3/2/train_output3/model.h5')

In [ ]:
with open('./experiments3/1/train_output3/history', 'rb') as file_history:
    history1 = pickle.load(file_history)

In [ ]:
with open('./experiments3/2/train_output3/history', 'rb') as file_history:
    history2 = pickle.load(file_history)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(14,8)
# ax.plot(history1['loss'][:])
ax.plot(history1['loss'][-1000:])
ax.plot(history1['val_loss'][-1000:])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(14,8)
# ax.plot(history1['loss'][:])
ax.plot(history2['loss'][-1000:])
ax.plot(history2['val_loss'][-1000:])

In [ ]:
for i,layer in enumerate(model1.layers):
    if i == 0:
        continue
    print(layer.get_weights()[0].shape, end=':\t')
    print(np.max(layer.get_weights()[0]), end='\t')
    print(np.min(layer.get_weights()[0]), end='\t| ')
    print(layer.get_weights()[0].shape, end=':\t')
    print(np.max(layer.get_weights()[1]), end='\t')
    print(np.min(layer.get_weights()[1]), end='\t')
    print(layer.get_weights()[1].shape)

In [ ]:
for i,layer in enumerate(model2.layers):
    if i == 0:
        continue
    print(layer.get_weights()[0].shape, end=':\t')
    print(np.max(layer.get_weights()[0]), end='\t')
    print(np.min(layer.get_weights()[0]), end='\t| ')
    print(layer.get_weights()[0].shape, end=':\t')
    print(np.max(layer.get_weights()[1]), end='\t')
    print(np.min(layer.get_weights()[1]), end='\t')
    print(layer.get_weights()[1].shape)

In [ ]:
n = 2
plt.plot(model1.layers[n].get_weights()[0][0], '.')
plt.plot(model2.layers[n].get_weights()[0][0], '.')